In [30]:
run = 'convolutional_21'

##### 00. Load Packages

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def get_available_cpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'CPU']

gpu_list = get_available_gpus()
cpu_list = get_available_cpus()

if len(gpu_list)>0:
    device_use = str(gpu_list[-1])
    print 'Using GPU : ' + device_use
else:
    device_use = str(cpu_list[-1])
    print 'Using CPU : ' + device_use

Using GPU : /gpu:1


In [3]:
%pylab inline


import seaborn as sns
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
import keras

import glob as glob
import cv2 as cv2
from tqdm import tqdm

Using TensorFlow backend.


In [5]:
from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dropout, Activation
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dense

from keras.layers.normalization import BatchNormalization

In [6]:
files_train = glob.glob('../01.data/extracted/images_training_rev1/*.jpg')
files_test = glob.glob('../01.data/extracted/images_test_rev1/*.jpg')

##### 00. Define functions

In [19]:
def get_image(file_name,size=(64,64),base_dir='../01.data/extracted/images_training_rev1/'):
    
    path = base_dir+str(file_name)+'.jpg'    
    x = cv2.imread(path)
    x = cv2.resize(x,size)
    return(x)

In [20]:
def get_labels(file_name):
    
    values = train_output.loc[np.int(file_name)].values
    
    return(values)

In [21]:
{
    'train':len(files_train),
    'test':len(files_test)
}

{'test': 79975, 'train': 61578}

In [22]:
y_path = '../01.data/extracted/training_solutions_rev1.csv'
train_output = pd.read_csv(y_path,index_col='GalaxyID')
train_output.sort_index(inplace=True)

In [24]:
n = 61578
image_size = (424,424)


shape_kernel = (2,2)
shape_pool = (2,2)

conv_activation = 'relu'
dense_activation = 'relu'

seed = 42
num_classes = 37
epochs = 1500

In [26]:
i = 1000

train_x = np.array([readImage(file_name,size=image_size)
                  for file_name in tqdm(train_output.index[:i],
                                        miniters=1000)
                 ])

train_y = np.array([get_labels(file_name)
                  for file_name in tqdm(train_output.index[:i],
                                        miniters=1000)
                 ])

train_y_expanded = np.expand_dims(np.expand_dims(train_y,1),1)

samples,img_rows, img_cols,img_channels = train_x.shape

100%|██████████| 1000/1000 [00:05<00:00, 177.08it/s]


In [27]:
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('train_y shape:', train_y_expanded.shape)

('train_x shape:', (1000, 424, 424, 3))
('train_y shape:', (1000, 37))
('train_y shape:', (1000, 1, 1, 37))


#### Convolutional

In [18]:
K.clear_session()

main_input = Input(shape=(img_rows,img_cols,img_channels), name='main_input')

x = Lambda(lambda x : x*1.0/255)(main_input)


x = BatchNormalization()(x)
x = Conv2D(filters=32, kernel_size=(2,2),
           data_format='channels_last', name='Conv-Input-a' )(x)
x = Conv2D(filters=32, kernel_size=(2,2),
           name='Conv-Input-b')(x)
x = Conv2D(filters=32, kernel_size=(2,2),
           name='Conv-Input-c')(x)
x = MaxPooling2D(pool_size=(4,4))(x)
x = Dropout(0.2)(x)


x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(2,2),
           name='Conv-1-a')(x)
x = Conv2D(filters=64, kernel_size=(2,2),
           name='Conv-1-b' )(x)
x = Conv2D(filters=64, kernel_size=(2,2),
           name='Conv-1-c' )(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3,3))(x)
x = Dropout(0.2)(x)


x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(2,2),
           name='Conv-2-a' )(x)
x = Conv2D(filters=128, kernel_size=(2,2),
           name='Conv-2-b' )(x)
x = Conv2D(filters=128, kernel_size=(2,2),
           name='Conv-2-c' )(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.2)(x)


x = BatchNormalization()(x)
x = Conv2D(filters=256, kernel_size=(2,2),
           name='Conv-3-a' )(x)
x = Conv2D(filters=256, kernel_size=(2,2),
           name='Conv-3-b' )(x)
x = Conv2D(filters=256, kernel_size=(2,2),
           name='Conv-3-c' )(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.2)(x)



x = BatchNormalization()(x)
x = Conv2D(filters=512, kernel_size=(2,2),
           name='Conv-4-a' )(x)
x = Conv2D(filters=512, kernel_size=(2,2),
           name='Conv-4-b' )(x)
x = Conv2D(filters=512, kernel_size=(2,2),
           name='Conv-4-c' )(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Dropout(0.2)(x)



x = BatchNormalization()(x)
x = Dense(1024,name='features')(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)

classifier = Dense(37,activation='linear',name='Output')(features)

model = Model(main_input,classifier,name='features')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 424, 424, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 424, 424, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 424, 424, 3)       12        
_________________________________________________________________
Conv-Input-a (Conv2D)        (None, 423, 423, 32)      416       
_________________________________________________________________
Conv-Input-b (Conv2D)        (None, 422, 422, 32)      4128      
_________________________________________________________________
Conv-Input-c (Conv2D)        (None, 421, 421, 32)      4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 105, 105, 32)      0         
__________

In [ ]:
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras_tqdm import TQDMNotebookCallback

tb = TensorBoard(
        log_dir='../tensorboard/'+run+'/',
        write_graph=True,
        write_images=True
    )

mc = ModelCheckpoint(filepath = '../05.model/'+run+'.h5',
                     save_best_only = True)

ec = EarlyStopping(monitor='val_loss',
                   patience=5,
                   mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=3,
                              min_lr=0.000000000001)

tqnc = TQDMNotebookCallback()

In [ ]:
batch_size = 150

train_steps = 2*train_x.shape[0]/batch_size
validation_steps = 0.1 * train_steps

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
                                rotation_range=180,
                                vertical_flip=True,
                                horizontal_flip=True,
                                data_format='channels_last',
                                
)

validation_datagen = ImageDataGenerator(
                                data_format='channels_last'
)

train_generator = train_datagen.flow(
                                    x=train_x,
                                    y=train_y_expanded[:i],
                                    batch_size=batch_size
)

validation_generator = validation_datagen.flow(
                                            x=train_x,
                                            y=train_y_expanded[:i],
                                            batch_size=batch_size
)

In [ ]:
with tf.device(device_use):

    model.compile(loss='mse',
                  optimizer=keras.optimizers.sgd(lr=1e-3)
                 )
    
    loss_history = model.fit_generator(
                                    generator=train_generator,
                                    validation_data=validation_generator,
                                    epochs=epochs,
                                    steps_per_epoch=train_steps,
                                    validation_steps=validation_steps,
                                    callbacks=[tb,mc,ec,reduce_lr,tqnc],
                                    verbose=1
                )


loss_df = pd.DataFrame(loss_history.history)
loss_df.to_csv('../03.plots/losses/augmented_loss_df'+run+'.csv',
                   index=False)

$$ \frac{1}{N} \sum_{i=0}^{N} Actual_{i} == Predicted_{i} $$

data_plot = loss_df.ix[1:,:]
sns.set(style='whitegrid',)
sns.plt.figure(figsize=(7.5,3.5))

plt.plot(data_plot.index,data_plot.loss,label='train')
plt.plot(data_plot.index,data_plot.val_loss,label='validation')
plt.legend()
plt.title('Train & Validation loss');

#### Training data

out_train = {}

with tf.device(device_use):
    
    for file_path in tqdm(files_train):

        galaxy_id = file_path.split('/')[-1].split('.')[0]
        galaxy_img = np.expand_dims(cv2.resize(cv2.imread(file_path),
                                               image_size),
                                    axis=0)
        galaxy_pred = model.predict(galaxy_img).flatten()

        out_train[galaxy_id] = galaxy_pred

columns = pd.read_csv(y_path,
                      index_col='GalaxyID',
                      nrows=0)

train_results = pd.DataFrame.from_dict(data = out_train,
                                      orient='index')
train_results.index.name = 'GalaxyID'
train_results.columns = columns.columns

#### Test data

In [ ]:
out = {}

with tf.device(device_use):  
    for file_path in tqdm(files_test):
        galaxy_id = file_path.split('/')[-1].split('.')[0]
        galaxy_img = np.expand_dims(cv2.resize(cv2.imread(file_path),
                                               image_size),
                                    axis=0)
        galaxy_pred = model.predict(galaxy_img).flatten()

        out[galaxy_id] = galaxy_pred

  0%|          | 0/79975 [00:00<?, ?it/s]

In [25]:
columns = pd.read_csv(y_path,
                      index_col='GalaxyID',
                      nrows=0)

test_results = pd.DataFrame.from_dict(data = out,
                                      orient='index')
test_results.index.name = 'GalaxyID'
test_results.columns = columns.columns

In [26]:
test_results.to_csv('../04.results/submission'+run+'.csv',
                    index_label='GalaxyID')